# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

  Using cached tensorflow-2.3.0-cp37-cp37m-win_amd64.whl (342.5 MB)
  Using cached numpy-1.18.5-cp37-cp37m-win_amd64.whl (12.7 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


# 1. Test Random Environment with OpenAI Gym

In [1]:
import gym 
import random

In [2]:
#env = gym.make('CartPole-v0')
env = gym.make('MountainCar-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [3]:
env.action_space

Discrete(3)

In [4]:
actions


3

In [ ]:
episodes = 50
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

# 2. Create a Deep Learning Model with Keras

In [6]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


In [7]:
def build_model(states, actions):
    #model =tensorflow.keras.Sequential()
    model=Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [8]:
model = build_model(states, actions)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                72        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 75        
Total params: 747
Trainable params: 747
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [10]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [11]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000*2, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [26]:
dqn = build_agent(model, actions)

In [27]:
#dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
#dqn.compile(metrics=['mae'])
#dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [28]:
dqn.fit(env, nb_steps=50000*2, visualize=False, verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 19:09 - reward: -1.0000

D:\Instalado\anaconda3\envs\tf\lib\site-packages\rl\memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 58s 6ms/step - reward: -1.0000
50 episodes - episode_reward: -200.000 [-200.000, -200.000] - loss: 2.264 - mae: 19.017 - mean_q: -28.178

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: -1.0000
50 episodes - episode_reward: -200.000 [-200.000, -200.000] - loss: 5.266 - mae: 32.170 - mean_q: -47.689

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: -1.0000 0s - rew
50 episodes - episode_reward: -200.000 [-200.000, -200.000] - loss: 6.879 - mae: 35.802 - mean_q: -53.144

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: -1.0000
50 episodes - episode_reward: -200.000 [-200.000, -200.000] - loss: 8.017 - mae: 39.107 - mean_q: -58.123

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 56s 6ms/step - reward: -1.0000
50 episodes - episode_reward: -20

In [29]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -200.000, steps: 200
Episode 2: reward: -144.000, steps: 144
Episode 3: reward: -138.000, steps: 138
Episode 4: reward: -140.000, steps: 140
Episode 5: reward: -137.000, steps: 137
Episode 6: reward: -200.000, steps: 200
Episode 7: reward: -145.000, steps: 145
Episode 8: reward: -200.000, steps: 200
Episode 9: reward: -148.000, steps: 148
Episode 10: reward: -200.000, steps: 200
Episode 11: reward: -137.000, steps: 137
Episode 12: reward: -200.000, steps: 200
Episode 13: reward: -148.000, steps: 148
Episode 14: reward: -200.000, steps: 200
Episode 15: reward: -138.000, steps: 138
Episode 16: reward: -136.000, steps: 136
Episode 17: reward: -200.000, steps: 200
Episode 18: reward: -200.000, steps: 200
Episode 19: reward: -136.000, steps: 136
Episode 20: reward: -200.000, steps: 200
Episode 21: reward: -200.000, steps: 200
Episode 22: reward: -138.000, steps: 138
Episode 23: reward: -141.000, steps: 141
Episode 24: reward: -200.000, steps: 

In [30]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: -137.000, steps: 137
Episode 2: reward: -138.000, steps: 138
Episode 3: reward: -141.000, steps: 141
Episode 4: reward: -143.000, steps: 143
Episode 5: reward: -200.000, steps: 200
Episode 6: reward: -138.000, steps: 138
Episode 7: reward: -149.000, steps: 149
Episode 8: reward: -142.000, steps: 142
Episode 9: reward: -140.000, steps: 140
Episode 10: reward: -136.000, steps: 136
Episode 11: reward: -140.000, steps: 140
Episode 12: reward: -200.000, steps: 200
Episode 13: reward: -136.000, steps: 136
Episode 14: reward: -137.000, steps: 137
Episode 15: reward: -137.000, steps: 137


# 4. Reloading Agent from Memory

In [42]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [43]:
del model
del dqn
del env

In [44]:
#env = gym.make('Pendulum-v0')
env = gym.make('CartPole-v0')

actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [45]:
dqn.load_weights('dqn_weights.h5f')

In [46]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...


KeyboardInterrupt: 